Application using altimetry data obtained from AVISO.
* Region of interest: Mediterranean Sea.
* Period of interest: May 2014.
* Satellite avaible: AltiKa, Cryosat, H2 and JASON-2. 

In [1]:
using divand
using NetCDF
using PyPlot
using Logging
Logging.configure(level=DEBUG);

# User inputs

## Domain of interest, resolution

In [13]:
lonmin, lonmax, latmin, latmax = -7., 40., 30., 48.
dx, dy, dt = .125, .125, 1.
lonr = lonmin:dx:lonmax;
latr = latmin:dy:latmax;
timer = 23510:dt:23529;

## File and directories

In [14]:
datadir = joinpath("../data/");
datafile = joinpath(datadir, "ADT_20140515_20140604.dat")
resultdir = joinpath("../results/divand/");
resultfile = joinpath(resultdir, "ADT_20140515_20140604_0pt125.nc");
info("Data file:\n", datafile)
info("Result file:\n", resultfile)

31-mars 18:20:56:INFO:root:Data file:
../data/ADT_20140515_20140604.dat
31-mars 18:20:56:INFO:root:Result file:
../results/divand/ADT_20140515_20140604_0pt125.nc


# Read data file (4-column, ascii)

In [4]:
function readAltimetryFile(fname)

    data = readlines(open(fname,"r"))
    nobs = length(data)

    lon = zeros(nobs)
    lat = zeros(nobs)
    time = zeros(nobs)
    field = zeros(nobs)

    for i in 1:nobs
        rec = split(data[i])
        lon[i] = parse(Float64,rec[1])
        lat[i] = parse(Float64,rec[2])
        time[i] = parse(Float64,rec[3])
        field[i] = parse(Float64,rec[4])
    end
    
    return lon, lat, time, field
end
info("Data read from file ", datafile)

31-mars 18:09:24:INFO:root:Data read from file ../data/ADT_20140515_20140604.dat


In [7]:
lon, lat, time, field = readAltimetryFile(datafile);
info("Reading ", length(lon), " data points")
info("Mean field value = ", mean(field))

31-mars 18:09:41:INFO:root:Reading 11371 data points
31-mars 18:09:41:INFO:root:Mean field value = -0.09550065957259697


Create mask and metrics using coordinate vectors

In [15]:
mask, (px, py, pt), (xi, yi, ti) = divand_rectdom(lonr, latr, timer);

## Analysis parameters

In [16]:
# correlation length
len = (1., 1., 4.);
# obs. error variance normalized by the background error variance
epsilon2 = 1;

# Perform interpolation

In [17]:
@time fi,s = divandrun(mask, (px, py, pt), (xi, yi, ti), (lon, lat, time), field, len, epsilon2; alphabc=2);

LoadError: OutOfMemoryError()

## Export results to netCDF

In [12]:
sz = size(mask)
dims = [NcDim("longitude",sz[1]), NcDim("latitude",sz[2]), NcDim("time",sz[3])];
nc = NetCDF.create(resultfile, NcVar("ADT", dims))
nc["ADT"][:,:, :] = fi
NetCDF.close(nc);
info("Written interpolated field in file:\n", resultfile)

31-mars 18:20:37:INFO:root:Written interpolated field in file:
../results/divand/ADT_20140515_20140604_0pt5.nc


# Plotting

Done in [another notebook in Python](../python/plot_AVISO_divand.ipynb) due to a problem with Julia / matplotlib.